##Set-Up Environment and Process Data

In [ ]:
#@title Basic setup
!pip install notebookjs
from notebookjs import execute_js

In [27]:
#@title Read in and process model data
import os
import json
import pandas as pd, numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
import requests

#set Parameters
MODEL_0 = 'OVSEG-SWNB' #@param {type:"string"} 
MODEL_1 = 'CSEG-SWNB' #@param {type:"string"} 
MODEL_FILE_PATH_0 = 'https://raw.githubusercontent.com/noellelaw/segvis/main/tools/model_results/clipens_results_pe.json' #@param {type:"string"}
MODEL_FILE_PATH_1 = 'https://raw.githubusercontent.com/noellelaw/segvis/main/tools/model_results/ovseg_swinb.json' #@param {type:"string"}
rng = np.random.default_rng(12345)
labels = ['unlabeled',
          'ego vehicle',
          'rect border',
          'out of roi',
          'static',
          'dynamic',
          'ground',
          'road',
          'sidewalk',
          'parking',
          'rail track',
          'building',
          'wall',
          'fence',
          'guard rail',
          'bridge',
          'tunnel',
          'pole',
          'pole group',
          'traffic light',
          'traffic sign',
          'vegetation',
          'terrain',
          'sky',
          'person',
          'rider',
          'car',
          'truck',
          'bus',
          'caravan',
          'trailer',
          'train',
          'motorcycle',
          'bicycle',]

real_labels = [
          'road',
          'sidewalk',
          'building',
          'wall',
          'fence',
          'pole',
          'traffic light',
          'traffic sign',
          'vegetation',
          'terrain',
          'sky',
          'person',
          'rider',
          'car',
          'truck',
          'bus',
          'train',
          'motorcycle',
          'bicycle',]


#Read in data
res_file = requests.get(MODEL_FILE_PATH_0)
json_data = json.loads(res_file.text)

confmat_0 = []
idx = 0
for i, pos in enumerate(json_data['confMatrix']):
  if labels[i] in real_labels:
    confmat_0.append([])
    for j, s in enumerate(pos):
      if labels[j] in real_labels:
          confmat_0[idx].append(s)
    idx += 1
confmat_0 = np.array(confmat_0) 


res_file = requests.get(MODEL_FILE_PATH_1)
json_data = json.loads(res_file.text)

confmat_1 = []
idx = 0
for i, pos in enumerate(json_data['confMatrix']):
  if labels[i] in real_labels:
    confmat_1.append([])
    for j, s in enumerate(pos):
      if labels[j] in real_labels:
          confmat_1[idx].append(s)
    idx += 1
confmat_1 = np.array(confmat_1) 


#Calculate and return IOU score for a particular label
def getIouScoreForLabel(label, confMatrix, num_classes):

    # the number of true positive pixels for this label
    # the entry on the diagonal of the confusion matrix
    tp = np.longlong(confMatrix[label,label])

    # the number of false negative pixels for this label
    # the row sum of the matching row in the confusion matrix
    # minus the diagonal entry
    fn = np.longlong(confMatrix[label,:].sum()) - tp

    # the number of false positive pixels for this labels
    # The column sum of the corresponding column in the confusion matrix
    # without the ignored rows and without the actual label of interest
    notIgnored = [l for l in range(num_classes) if not l==label]
    fp = np.longlong(confMatrix[notIgnored,label].sum())

    # the denominator of the IOU score
    denom = (tp + fp + fn)
    if denom == 0:
        return float('nan')

    # return IOU
    return float(tp) / denom



# Parse data
data = {'road': { 
          'color': '#804080' 
          },
      'sidewalk': { 
          'color': '#F423E8'
          },
      'building': { 
          'color': '#464646'
          },
      'wall': { 
          'color': '#66669C'
          },
      'fence': { 
          'color': '#BE9999'
          },
      'pole': { 
          'color': '#999999'
          },
      'traffic light': { 
          'color': '#FAAA1E'
          },
      'traffic sign': { 
          'color': '#DCDC00'
          },
      'vegetation': { 
          'color': '#6B8E23'
          },
      'terrain': { 
          'color': '#98FB98'
          },
      'sky': { 
          'color': '#4682B4'
          },
      'person': { 
          'color': '#AA143C'
          },
      'rider': { 
          'color': '#FF0000'
          },
      'car': { 
          'color': '#00006E'
          },
      'truck': { 
          'color': '#000044'
          },
      'bus': { 
          'color': '#003C64'
          },
      'train': { 
          'color': '#008064'
          },
      'motorcycle': { 
          'color': '#0000E8'
          },
      'bicycle': { 
          'color': '#770B20'
          }}

for n in data.keys():
  if n in real_labels:
    n_index = real_labels.index(n)
    data[n]['index'] = n_index
    total_0 = float(confmat_0[:,n_index].sum())
    total_1 = float(confmat_1[:,n_index].sum())
    data[n]['miou_0'] = np.round(getIouScoreForLabel(n_index, confmat_0, 19),3)
    data[n]['miou_1'] = np.round(getIouScoreForLabel(n_index, confmat_1, 19),3)
    data[n]['model_0'] = MODEL_0
    data[n]['model_1'] = MODEL_1
    data[n]['data'] = []
    data[n]['classes'] = []
    for m in data.keys():
      if m in labels:
        m_index = real_labels.index(m)
        if total_0 == 0.0:
          preds = {'preds_0': float(0),
                  'name_0': m,
                  'class_total_0': int(confmat_0[m_index, n_index]),
                  'preds_1': float(confmat_1[m_index, n_index] / total_1),
                  'name_1': m,
                  'class_total_1': int(confmat_1[m_index, n_index]),
                   'model_0': MODEL_0,
                   'model_1': MODEL_1
                   }
        elif total_1 == 0.0:
          preds = {'preds_0': float(confmat_0[m_index, n_index] / total_0),
                  'name_0': m,
                  'class_total_0': int(confmat_0[m_index, n_index]),
                  'preds_1': float(0),
                  'name_1': m,
                  'class_total_1': int(confmat_1[m_index, n_index]),
                   'model_0': MODEL_0,
                   'model_1': MODEL_1}
        else:
          preds = {'preds_0': float(confmat_0[m_index, n_index] / total_0),
                  'name_0': m,
                  'class_total_0': int(confmat_0[m_index, n_index]),
                  'preds_1': float(confmat_1[m_index, n_index] / total_1),
                  'name_1': m,
                  'class_total_1': int(confmat_1[m_index, n_index]),
                   'model_0': MODEL_0,
                   'model_1': MODEL_1}
        data[n]['data'].append(preds)

##Plot Triangles using D3 File

In [28]:
#@title Plot Triangles
import random


code_js  = requests.get("https://raw.githubusercontent.com/noellelaw/segvis/main/tools/model_results/semseg.js").text
d3_path = "https://d3js.org/d3.v7.min.js"

def python_callback( data ):

  global b
  b = data

  return {'n': random.random(), 'y': 'VisML' }

execute_js(library_list=[d3_path, code_js], 
           main_function="draw_full_linegraph", 
           data_dict={'data': data},
           callbacks={'python_callback': python_callback})